In [4]:
import argparse
from pytubefix import YouTube
from pytubefix.cli import on_progress
from googleExample import youtube_search
import os
from extract_audio import extract_audio
from readVideosTitle import leggi_titoli_video

# Dove salvare
SAVE_PATH = os.path.expanduser("~/Desktop/Facial-Emotion-Recognition\ base")

def create_link(videoIds):
	links = []
	for videoId in videoIds:
		print(f"Processing video ID: {videoId}")
		link = f"https://www.youtube.com/watch?v={videoId}"
		links.append(link)
	return links

def download_youtube_video(url, save_path='.'):
	try:
		print(f"Attempting to download video from URL: {url}")
		yt = YouTube(url, on_progress_callback = on_progress)

		ys = yt.streams.get_highest_resolution()
		ys.download(output_path='./videos')

	except Exception as e:
		print(f"Errore durante il download: {e}")

if __name__ == "__main__":
	# Create an argument parser
	parser = argparse.ArgumentParser()
	parser.add_argument('--q', help='Search term', default='Google')
	parser.add_argument('--max-results', help='Max results', default=5, type=int)

	# Instead of parsing command line arguments, set them in a dictionary
	args_dict = {'q': 'Chris Evans interview', 'max_results': 5}  # Replace with your desired values
	args = parser.parse_args(args=[])  # Initialize args
	args.__dict__.update(args_dict)  # Update args with your values
	os.makedirs('./videos', exist_ok=True)
	os.makedirs('./audio', exist_ok=True)
	os.makedirs('./transcriptions', exist_ok=True)

	videoIds = youtube_search(args)
	print(f"Video IDs: {videoIds}")

	# Links dei video da scaricare
	links = create_link(videoIds)
	print(f"Links: {links}")

	# Scarica i video dalla lista
	for link in links:
		download_youtube_video(link, SAVE_PATH)
	video_titles = leggi_titoli_video()

	for video_title in video_titles:
		extract_audio('./videos/' + video_title.replace(' ', '_') + '.mp4', './audio/' + video_title + '.wav')

Video IDs: ['2dPogeNhEoQ', 'NW69107Acrs', 'v7ggWM0ef-M', '9wMizV1KG-M', 'f-O_F84JcSI']
Processing video ID: 2dPogeNhEoQ
Processing video ID: NW69107Acrs
Processing video ID: v7ggWM0ef-M
Processing video ID: 9wMizV1KG-M
Processing video ID: f-O_F84JcSI
Links: ['https://www.youtube.com/watch?v=2dPogeNhEoQ', 'https://www.youtube.com/watch?v=NW69107Acrs', 'https://www.youtube.com/watch?v=v7ggWM0ef-M', 'https://www.youtube.com/watch?v=9wMizV1KG-M', 'https://www.youtube.com/watch?v=f-O_F84JcSI']
Attempting to download video from URL: https://www.youtube.com/watch?v=2dPogeNhEoQ
Attempting to download video from URL: https://www.youtube.com/watch?v=NW69107Acrs
Attempting to download video from URL: https://www.youtube.com/watch?v=v7ggWM0ef-M
Attempting to download video from URL: https://www.youtube.com/watch?v=9wMizV1KG-M
Attempting to download video from URL: https://www.youtube.com/watch?v=f-O_F84JcSI
MoviePy - Writing audio in ./audio/Chris_Evans_OPENS_UP_About_His_‘Deadpool_&_Wolverine’_C

MoviePy - Done.
MoviePy - Writing audio in ./audio/Chris_Evans_The_Puppy_Interview.wav


MoviePy - Done.
MoviePy - Writing audio in ./audio/Chris_Evans_Answers_the_Webs_Most_Searched_Questions__WIRED.wav


MoviePy - Done.
MoviePy - Writing audio in ./audio/The_Avengers_Interviews_Chris_Hemsworth_and_Chris_Evans.wav


MoviePy - Done.
MoviePy - Writing audio in ./audio/Chris_Evans_was_Hazed_by_Paul_Rudd_in_Their_Fantasy_Football_League__The_Tonight_Show.wav


MoviePy - Done.


In [6]:
from expression_ssd_detect import FER_live_cam
from readVideosTitle import leggi_titoli_video
from transcribe import transcribe_audio
import multiprocessing

if __name__ == "__main__":
	titoli = leggi_titoli_video()

	for titolo in titoli:
		p1 = multiprocessing.Process(target=FER_live_cam, args=(titolo,))
		p2 = multiprocessing.Process(target=transcribe_audio, args=(titolo,))

		# Avvio dei processi
		p1.start()
		p2.start()

		# Attendere il completamento dei processi
		p1.join()
		p2.join()



[swscaler @ 0x1187d0000] [swscaler @ 0x1282e8000] No accelerated colorspace conversion found from yuv420p to rgb24.
[swscaler @ 0x1187d0000] [swscaler @ 0x1282f8000] No accelerated colorspace conversion found from yuv420p to rgb24.
[swscaler @ 0x1187d0000] [swscaler @ 0x128308000] No accelerated colorspace conversion found from yuv420p to rgb24.
[swscaler @ 0x1187d0000] [swscaler @ 0x128318000] No accelerated colorspace conversion found from yuv420p to rgb24.
[swscaler @ 0x1187d0000] [swscaler @ 0x128328000] No accelerated colorspace conversion found from yuv420p to rgb24.
[swscaler @ 0x1187d0000] [swscaler @ 0x128338000] No accelerated colorspace conversion found from yuv420p to rgb24.
[swscaler @ 0x1187d0000] [swscaler @ 0x128348000] No accelerated colorspace conversion found from yuv420p to rgb24.
[swscaler @ 0x1187d0000] [swscaler @ 0x128358000] No accelerated colorspace conversion found from yuv420p to rgb24.
[swscaler @ 0x1187d0000] [swscaler @ 0x128368000] No accelerated colorsp

Detected language in segment 0: en
Detected language in segment 1: en
Detected language in segment 2: en
Detected language in segment 3: en
Detected language in segment 4: en
Detected language in segment 5: en
Detected language in segment 6: en
Detected language in segment 7: en
Detected language in segment 8: en
Detected language in segment 9: en
Detected language in segment 10: en
Detected language in segment 11: en
Detected language in segment 12: en
Detected language in segment 13: en
Detected language in segment 14: en
Detected language in segment 15: en
Detected language in segment 16: en
Detected language in segment 17: en
priors nums:4420
FPS: 30.6
FPS: 22.6
FPS: 92.2
FPS: 100.5
FPS: 94.8
FPS: 101.0
FPS: 97.8
FPS: 107.3
FPS: 104.7
FPS: 95.8
FPS: 99.3
FPS: 103.8
FPS: 100.5
FPS: 83.6
FPS: 51.4
FPS: 82.1
FPS: 86.1
FPS: 110.3
FPS: 113.8
FPS: 123.7
FPS: 114.0
FPS: 109.0
FPS: 125.5
FPS: 108.2
FPS: 91.0
FPS: 99.2
FPS: 107.4
FPS: 112.7
FPS: 103.1
FPS: 112.4
FPS: 102.5
FPS: 110.6
FPS: 1

Process Process-9:
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/Users/francesconicotra/Desktop/Facial-Emotion-Recognition base/expression_ssd_detect.py", line 250, in FER_live_cam
    resize_frame = cv2.resize(
                   ^^^^^^^^^^^
cv2.error: OpenCV(4.9.0) /Users/xperience/GHA-OpenCV-Python2/_work/opencv-python/opencv-python/opencv/modules/imgproc/src/resize.cpp:4152: error: (-215:Assertion failed) !ssize.empty() in function 'resize'



FPS: 111.2
FPS: 113.5
FPS: 103.8
FPS: 109.7
FPS: 110.5
FPS: 109.1
FPS: 108.2
FPS: 108.8
FPS: 113.8
FPS: 112.4
FPS: 106.3
FPS: 108.7
FPS: 105.5
FPS: 113.2
FPS: 108.6
FPS: 110.7
FPS: 94.7
FPS: 97.3
FPS: 98.2
FPS: 109.3
FPS: 105.5
FPS: 107.4
FPS: 103.5
FPS: 100.9
FPS: 106.4
FPS: 112.1
FPS: 100.5
FPS: 106.4
FPS: 105.6
FPS: 105.8
FPS: 112.1
FPS: 108.1
FPS: 108.1
FPS: 113.1
FPS: 113.6
FPS: 107.3
FPS: 112.4
FPS: 102.1
FPS: 97.2
FPS: 99.3
FPS: 108.3
FPS: 106.6
FPS: 109.6
FPS: 109.9
FPS: 107.0
FPS: 104.3
FPS: 113.1
FPS: 103.8
FPS: 106.1
FPS: 105.5
FPS: 108.5
FPS: 114.9
FPS: 111.9
FPS: 107.9
FPS: 112.3
FPS: 110.8
FPS: 112.6
FPS: 110.8
FPS: 102.8
FPS: 106.7
FPS: 113.1
FPS: 109.0
FPS: 107.2
FPS: 106.2
FPS: 110.5
FPS: 100.1
FPS: 66.4
FPS: 107.3
FPS: 68.5
FPS: 105.0
FPS: 68.8
FPS: 107.4
FPS: 68.3
FPS: 108.5
FPS: 69.0
FPS: 109.6
FPS: 70.1
FPS: 106.3
FPS: 66.7
FPS: 101.2
FPS: 66.3
FPS: 107.8
FPS: 106.0
FPS: 104.9
FPS: 109.3
FPS: 109.1
FPS: 114.1
FPS: 103.0
FPS: 110.3
FPS: 107.1
FPS: 104.3
FPS: 102.4
F

[swscaler @ 0x128930000] [swscaler @ 0x128940000] No accelerated colorspace conversion found from yuv420p to rgb24.
[swscaler @ 0x128930000] [swscaler @ 0x149a68000] No accelerated colorspace conversion found from yuv420p to rgb24.
[swscaler @ 0x128930000] [swscaler @ 0x149a78000] No accelerated colorspace conversion found from yuv420p to rgb24.
[swscaler @ 0x128930000] [swscaler @ 0x149a88000] No accelerated colorspace conversion found from yuv420p to rgb24.
[swscaler @ 0x128930000] [swscaler @ 0x149a98000] No accelerated colorspace conversion found from yuv420p to rgb24.
[swscaler @ 0x128930000] [swscaler @ 0x128950000] No accelerated colorspace conversion found from yuv420p to rgb24.
[swscaler @ 0x128930000] [swscaler @ 0x128960000] No accelerated colorspace conversion found from yuv420p to rgb24.
[swscaler @ 0x128930000] [swscaler @ 0x128970000] No accelerated colorspace conversion found from yuv420p to rgb24.
[swscaler @ 0x128930000] [swscaler @ 0x128980000] No accelerated colorsp

...

Process Process-15:
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/Users/francesconicotra/Desktop/Facial-Emotion-Recognition base/expression_ssd_detect.py", line 250, in FER_live_cam
    resize_frame = cv2.resize(
                   ^^^^^^^^^^^
cv2.error: OpenCV(4.9.0) /Users/xperience/GHA-OpenCV-Python2/_work/opencv-python/opencv-python/opencv/modules/imgproc/src/resize.cpp:4152: error: (-215:Assertion failed) !ssize.empty() in function 'resize'



priors nums:4420
FPS: 27.0
FPS: 60.4
FPS: 87.4
FPS: 101.8
FPS: 98.3
FPS: 98.2
FPS: 95.6
FPS: 101.3
FPS: 101.9
FPS: 97.0
FPS: 79.0
FPS: 62.5
FPS: 79.9
FPS: 104.6
FPS: 107.6
FPS: 110.1
FPS: 118.5
FPS: 110.2
FPS: 119.1
FPS: 126.7
FPS: 106.7
FPS: 98.8
FPS: 105.7
FPS: 96.8
FPS: 92.7
FPS: 101.0
FPS: 95.6
FPS: 79.4
FPS: 61.6
FPS: 85.3
FPS: 64.8
FPS: 88.9
FPS: 65.9
FPS: 82.5
FPS: 78.4
FPS: 86.3
FPS: 83.7
FPS: 82.7
FPS: 74.9
FPS: 84.0
FPS: 97.3
FPS: 94.0
FPS: 92.0
FPS: 86.1
FPS: 82.7
FPS: 71.8
FPS: 84.8
FPS: 88.4
FPS: 84.6
FPS: 84.5
FPS: 70.4
FPS: 84.4
FPS: 76.9
FPS: 89.0
FPS: 85.3
FPS: 91.3
FPS: 87.7
FPS: 86.8
FPS: 81.8
FPS: 82.1
FPS: 84.0
FPS: 85.9
FPS: 89.6
FPS: 88.8
FPS: 84.8
FPS: 85.7
FPS: 82.0
FPS: 83.6
FPS: 87.8
FPS: 86.1
FPS: 86.5
FPS: 80.3
FPS: 78.6
FPS: 89.1
FPS: 86.7
FPS: 82.4
FPS: 55.3
FPS: 84.8
FPS: 55.2
FPS: 87.8
FPS: 55.4
FPS: 63.5
FPS: 45.1
FPS: 82.6
FPS: 53.7
FPS: 34.5
FPS: 75.0
FPS: 48.1
FPS: 35.4
FPS: 81.6
FPS: 52.6
FPS: 76.1
FPS: 87.0
FPS: 55.8
FPS: 40.4
FPS: 89.6
FPS: 53.7


[swscaler @ 0x15a4f0000] [swscaler @ 0x140fd0000] No accelerated colorspace conversion found from yuv420p to rgb24.
[swscaler @ 0x15a4f0000] [swscaler @ 0x140fe0000] No accelerated colorspace conversion found from yuv420p to rgb24.
[swscaler @ 0x15a4f0000] [swscaler @ 0x140ff0000] No accelerated colorspace conversion found from yuv420p to rgb24.
[swscaler @ 0x15a4f0000] [swscaler @ 0x141000000] No accelerated colorspace conversion found from yuv420p to rgb24.
[swscaler @ 0x15a4f0000] [swscaler @ 0x141010000] No accelerated colorspace conversion found from yuv420p to rgb24.
[swscaler @ 0x15a4f0000] [swscaler @ 0x141020000] No accelerated colorspace conversion found from yuv420p to rgb24.
[swscaler @ 0x15a4f0000] [swscaler @ 0x141030000] No accelerated colorspace conversion found from yuv420p to rgb24.
[swscaler @ 0x15a4f0000] [swscaler @ 0x1424a0000] No accelerated colorspace conversion found from yuv420p to rgb24.
[swscaler @ 0x15a4f0000] [swscaler @ 0x1424b0000] No accelerated colorsp

Detected language in segment 0: en
Detected language in segment 1: en
Detected language in segment 2: en
Detected language in segment 3: en
Detected language in segment 4: en
Detected language in segment 5: en
priors nums:4420
FPS: 21.7
FPS: 99.1
FPS: 97.9
FPS: 94.0
FPS: 60.6
FPS: 100.7
FPS: 60.9
FPS: 102.2
FPS: 65.3
FPS: 96.4
FPS: 59.1
FPS: 89.6
FPS: 54.7
FPS: 79.6
FPS: 47.6
FPS: 86.4
FPS: 54.2
FPS: 108.9
FPS: 68.1
FPS: 122.4
FPS: 73.5
FPS: 124.8
FPS: 77.2
FPS: 103.0
FPS: 66.4
FPS: 123.4
FPS: 79.6
FPS: 104.3
FPS: 65.6
FPS: 100.1
FPS: 63.2
FPS: 106.6
FPS: 66.0
FPS: 123.6
FPS: 77.0
FPS: 120.2
FPS: 73.6
FPS: 135.5
FPS: 80.9
FPS: 105.2
FPS: 64.9
FPS: 102.3
FPS: 64.4
FPS: 73.6
FPS: 37.6
FPS: 61.5
FPS: 32.2
FPS: 92.0
FPS: 59.2
FPS: 81.8
FPS: 53.5
FPS: 91.6
FPS: 59.6
FPS: 89.3
FPS: 56.7
FPS: 95.5
FPS: 60.2
FPS: 63.9
FPS: 45.2
FPS: 87.6
FPS: 57.0
FPS: 90.4
FPS: 58.2
FPS: 87.7
FPS: 53.0
FPS: 78.4
FPS: 52.5
FPS: 96.0
FPS: 60.6
FPS: 104.4
FPS: 65.3
FPS: 99.8
FPS: 61.1
FPS: 97.1
FPS: 63.4
FPS: 91

FPS: 114.0
FPS: 73.2
FPS: 109.9
FPS: 68.6
FPS: 108.0
FPS: 70.1
FPS: 106.0
FPS: 69.3
FPS: 113.2
FPS: 71.9
FPS: 104.2
FPS: 67.4
FPS: 106.7
FPS: 69.2
FPS: 116.2
FPS: 72.7
FPS: 113.1
FPS: 72.7
FPS: 103.7
FPS: 67.1
FPS: 105.5
FPS: 69.7
FPS: 107.1
FPS: 69.7
FPS: 112.1
FPS: 72.1
FPS: 111.7
FPS: 69.8
FPS: 117.3
FPS: 72.8
FPS: 105.4
FPS: 67.2
FPS: 111.2
FPS: 70.8
FPS: 114.3
FPS: 71.6
FPS: 107.0
FPS: 70.9
FPS: 107.6
FPS: 69.3
FPS: 112.4
FPS: 71.8
FPS: 109.8
FPS: 70.7
FPS: 110.6
FPS: 70.3
FPS: 109.3
FPS: 70.1
FPS: 105.5
FPS: 69.7
FPS: 105.3
FPS: 69.5
FPS: 112.3
FPS: 69.3
FPS: 117.2
FPS: 72.7
FPS: 113.7
FPS: 71.8
FPS: 105.0
FPS: 69.4
FPS: 118.6
FPS: 73.9
FPS: 111.4
FPS: 70.5
FPS: 110.4
FPS: 113.3
FPS: 108.6
FPS: 101.9
FPS: 110.2
FPS: 110.0
FPS: 111.4
FPS: 110.8
FPS: 112.8
FPS: 111.0
FPS: 111.7
FPS: 109.7
FPS: 114.6
FPS: 107.4
FPS: 98.6
FPS: 110.5
FPS: 99.6
FPS: 108.0
FPS: 107.6
FPS: 108.0
FPS: 113.0
FPS: 112.3
FPS: 119.1
FPS: 111.3
FPS: 104.7
FPS: 109.0
FPS: 107.5
FPS: 105.4
FPS: 107.7
FPS: 110.2
